Part Four:
*   Split Each Audio into n(=5) Pieces (for LSTM Model)
*   Extract Feature Values of Each Slice & Their Average
*   Store Data Extracted in Previous Step in a Pickle File


**Import Headers**

In [0]:
import os
import librosa
import numpy as np
import pickle as pkl
!pip install pydub
from pydub import AudioSegment
import shutil #library by which parent folder can be deleted irrespective of its contents
import warnings
warnings.filterwarnings('ignore')

**Function to Return the List of Genre Folders**

In [0]:
def genre_lists(data_path):
  files = os.listdir(data_path)
  genres = []
  for g in files:
    if(g.endswith("mf")):
      continue
    else:
      genres.append(g)
  return(genres)

**Function to Return Extracted Features**

In [0]:
def extract_feature(audio_file):
    extracted_features = []    
    y, sr = librosa.load(audio_file)
    chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
    rmse = librosa.feature.rms(y=y)
    spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
    spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
    rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
    zcr = librosa.feature.zero_crossing_rate(y)
    mfcc = librosa.feature.mfcc(y=y, sr=sr)    
    extracted_features.append(np.mean(chroma_stft))
    extracted_features.append(np.mean(rmse)) 
    extracted_features.append(np.mean(spec_cent))
    extracted_features.append(np.mean(spec_bw))
    extracted_features.append(np.mean(rolloff))
    extracted_features.append(np.mean(zcr))    
    for e in mfcc:
        extracted_features.append(np.mean(e))
    return extracted_features

**Split Audio & Extract Features of Each Slice**

In [0]:
PATH = "/content/drive/My Drive/Project/Data/Working_Data/genres"
no_of_splits = 5
MUSIC_DATA = []
genre_list = []
genres = genre_lists(PATH)
for g in genres:
    Y = []
    c=1;
    print("Genre",g,"started.")
    song_list = os.listdir(PATH + "/" + g)
    for s in song_list:
        X = []
        os.mkdir("/content/TEMP")
        sa = PATH + "/" + g + "/" + s
        audio = AudioSegment.from_wav(sa)
        length = len(audio)
        split_at = length//no_of_splits
        st = 0
        for i in range(no_of_splits):
            et = st + split_at
            if(i < no_of_splits - 1):
                modified_audio = audio[st:et]
            else:
                modified_audio = audio[st:]
            st = et
            name = "/content/TEMP" + g + "_modified_" + str(i+1) + "_" + s
            modified_audio.export(name,format = "wav")
            extracted_features = extract_feature(name)
            X.append(extracted_features)
        shutil.rmtree("/content/TEMP") #removes tree of folders & subfolders
        print("Song",str(c),"completed.")
        c+=1
        X = np.array(X)
        MUSIC_DATA.append(X)
        genre_list.append(g)
MUSIC_DATA = np.array(MUSIC_DATA)
genre_list = np.array(genre_list)
print("The shape is:", MUSIC_DATA.shape)

**Write Feature & Genre Arrays into Pickle Files**

In [0]:
with open('/content/drive/My Drive/Project/Data/Working_Data/DATA_PREPROCESS_LSTM_5.pkl','wb') as f:
    pkl.dump(MUSIC_DATA, f)
with open('/content/drive/My Drive/Project/Data/Working_Data/GENRE_LIST_LSTM_5.pkl','wb') as f:
    pkl.dump(genre_list, f)
with open('/content/drive/My Drive/Project/Data/Working_Data/DATA_PREPROCESS_LSTM_5.pkl','rb') as f:
    X = pkl.load(f)
with open('/content/drive/My Drive/Project/Data/Working_Data/GENRE_LIST_LSTM_5.pkl','rb') as f:
    Y = pkl.load(f)
print("The shape of the X - data (Music) is:",X.shape)
print("The shape of the Y - data (Genre) is:",Y.shape)
print("Part Four Successful!")